In [1]:
# creates snorkel session, according to the tutorial
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
from snorkel import SnorkelSession
session = SnorkelSession()


snorkel_conn_string sqlite:////Users/xinq/Desktop/BAR-code/snorkel.db
if snorkel_postgres? False


In [7]:
import os

print(os.getcwd())

/Users/xinq/Desktop/BAR-code


In [9]:
from snorkel.parser import TSVDocPreprocessor # I used a TSV file to store the wikipedia articles
doc_preprocessor = TSVDocPreprocessor('~/Desktop/BAR-code/Wikipedia/BioArticles.tsv', max_docs=20000)
from snorkel.parser.spacy_parser import Spacy
from snorkel.parser import CorpusParser

corpus_parser = CorpusParser(parser=Spacy())
%time corpus_parser.apply(doc_preprocessor, count=20000, clear=False)

Running UDF...
snorkel_conn_string sqlite:////Users/xinq/Desktop/BAR-code/snorkel.db
if snorkel_postgres? False
[=                                       ] 0%

OSError: File or directory not found: ~/Desktop/BAR-code/Wikipedia/BioArticles.tsv

In [3]:
from snorkel.candidates import Ngrams, CandidateExtractor
from snorkel.matchers import PersonMatcher, RegexMatchEach
from snorkel.models import candidate_subclass

# my candidates are sentences that has the following components:
# subject - the animal/organ/protein etc. that solves a problem. For example, in "the bat uses sonar so it could navigate in the dark",
# Bat is the animal that solves the navigation problem
# verb - the solution to the problem. For example, in "the bat uses sonar so it could navigate in the dark", the use of sonar is the 
# solution to the navigation problem.
# a relative clause - the problem. For example, in "the bat uses sonar so it could navigate in the dark", 
# "it could navigate in the dark" is a relative clause and the problem.
# I used snorkel ability to extract candidates by their dep_labels to find sentences with the 3 components
# (using RegexMatchEach, which is used to search specific strings)
problem_solver = candidate_subclass('problem_solver', ['solver', 'solution', 'problem'])
ngrams         = Ngrams(n_max=7)
solver = RegexMatchEach(rgx='nsubj',attrib='dep_labels', longest_match_only=True)
solution = RegexMatchEach(rgx='ROOT',attrib='dep_labels', longest_match_only=True)
problem = RegexMatchEach(rgx='relcl',attrib='dep_labels', longest_match_only=True)
cand_extractor = CandidateExtractor(problem_solver, [ngrams, ngrams, ngrams], [solver, solution, problem])

In [4]:
# from here, same as the tutorial

from snorkel.models import Document
docs = session.query(Document).order_by(Document.name).all()

train_sents = set()
dev_sents   = set()
test_sents  = set()

for i, doc in enumerate(docs):
    for s in doc.sentences:
        if i % 10 == 8:
            dev_sents.add(s)
        elif i % 10 == 9:
            test_sents.add(s)
        else:
            train_sents.add(s)

In [5]:
cand_extractor.apply(dev_sents, split=1)

Clearing existing...
Running UDF...
[========================================] 100%



In [6]:
%%time
for i, sents in enumerate([train_sents, dev_sents, test_sents]):
    cand_extractor.apply(sents, split=i)
    print("Number of candidates:", session.query(problem_solver).filter(problem_solver.split == i).count())

Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 228663)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 31097)
Clearing existing...
Running UDF...
[========================================] 100%

('Number of candidates:', 28996)
CPU times: user 2h 15min 4s, sys: 6min 52s, total: 2h 21min 56s
Wall time: 3h 17min 11s
